In [3]:
import nltk
import string

# input text
text = "Natural language %processing? is a field of? artificial intelligence, that deals with the interaction@ between computers and human (natural) language."

tokens = nltk.word_tokenize(text)

filtered_tokens = [token for token in tokens if token not in string.punctuation]


print(" ".join(filtered_tokens))


Natural language processing is a field of artificial intelligence that deals with the interaction between computers and human natural language


In [1]:
import pandas as pd
import nltk
import string

def removePunctuation(df, column):
    # Tokenize the text in the specified column for each row in the DataFrame
    df[column] = df[column].apply(lambda text: nltk.word_tokenize(text))

    # Remove punctuation from tokens for each row
    df[column] = df[column].apply(lambda tokens: [token for token in tokens if token not in string.punctuation])

    # Join the tokens back into text for each row
    df[column] = df[column].apply(lambda tokens: " ".join(tokens))

    return df

# Example usage:
# Create a sample DataFrame
data = {'text_column': ["Natural language %processing? is a field of? artificial intelligence, that deals with the interaction@ between computers and human (natural) language.",
                        "Another sentence with punctuation!"]}
df = pd.DataFrame(data)

# Call the function to remove punctuation from the 'text_column'
df_cleaned = removePunctuation(df, 'text_column')

print(df_cleaned)


                                         text_column
0  Natural language processing is a field of arti...
1                  Another sentence with punctuation


In [5]:
data_path = "dataset\\twitter_training.csv"
temp_df = pd.read_csv(data_path)

# Check the current data type of the column
print("Current data type of 'Borderlands' column:", temp_df["Borderlands"].dtype)

# Convert the data type of the column to string
temp_df["Borderlands"] = temp_df["Borderlands"].astype(str)

# Check the updated data type of the column
print("Updated data type of 'Borderlands' column:", temp_df["Borderlands"].dtype)


Current data type of 'Borderlands' column: object
Updated data type of 'Borderlands' column: object


In [6]:
temp_df.columns

Index(['2401', 'Borderlands', 'Positive',
       'im getting on borderlands and i will murder you all ,'],
      dtype='object')

In [7]:
import pandas as pd
import nltk
import string

# Read the CSV file into a DataFrame
data_path = "dataset\\twitter_training.csv"
temp_df = pd.read_csv(data_path)
# Rename the column to "new"
temp_df.rename(columns={"im getting on borderlands and i will murder you all ,": "new"}, inplace=True)

# Define a function to remove punctuation using NLTK
def remove_punctuation(text):
    if isinstance(text, str):
        tokenizer = nltk.RegexpTokenizer(r'\w+')
        tokens = tokenizer.tokenize(text)
        return ' '.join(tokens)
    else:
        return text

# Apply the function to the "new" column
temp_df["new"] = temp_df["new"].apply(remove_punctuation)

# Print the first few rows of the DataFrame to verify the changes
print(temp_df.head())


   2401  Borderlands  Positive  \
0  2401  Borderlands  Positive   
1  2401  Borderlands  Positive   
2  2401  Borderlands  Positive   
3  2401  Borderlands  Positive   
4  2401  Borderlands  Positive   

                                                 new  
0  I am coming to the borders and I will kill you...  
1  im getting on borderlands and i will kill you all  
2  im coming on borderlands and i will murder you...  
3  im getting on borderlands 2 and i will murder ...  
4  im getting into borderlands and i can murder y...  


In [14]:
from nltk.stem import WordNetLemmatizer
# nltk.download("wordnet")
# nltk.download("omw-1.4")
def lemmatize(text):
    wnl = WordNetLemmatizer()
    if isinstance(text, str):
        text_tokens = nltk.word_tokenize(text)
        lemmatized_tokens = [wnl.lemmatize(word, pos="v") for word in text_tokens]
        return " ".join(lemmatized_tokens)

    
lemmatize("Python programmers often tend like programming in")

'Python programmers often tend like program in'

In [1]:
import pandas as pd
import cudf
from cudf import DataFrame
from dask_cuda import LocalCUDACluster
from dask.distributed import Client
import cunlp

# Initialize ROCm
cunlp.init(backend="rocm")

# Create a Dask-CUDA cluster and client
cluster = LocalCUDACluster()
client = Client(cluster)

def remove_stopwords(df, column):
    # Convert pandas DataFrame to cuDF DataFrame
    gdf = cudf.from_pandas(df)

    # Remove stop words using cuNLP
    stopwords = cunlp.stopwords("en")
    gdf[column] = gdf[column].str.remove_stopwords(stopwords=stopwords)

    # Convert back to pandas DataFrame
    processed_df = gdf.to_pandas()
    return processed_df

if __name__ == '__main__':
    # Path to the dataset
    data_path = "dataset\\WAZE_REVIEWS.csv"

    # Load your dataframe
    df = pd.read_csv(data_path)

    # Scatter the DataFrame across the Dask-CUDA cluster
    ddf = client.scatter(df)

    # Process the DataFrame in parallel on the GPU
    processed_ddf = ddf.map_partitions(remove_stopwords, column="review_text", meta=df)

    # Gather the processed DataFrame back to the main process
    processed_df = client.gather(processed_ddf)

    print(processed_df)

ModuleNotFoundError: No module named 'cudf'

In [8]:
import time
data_path = "dataset\\Corona_NLP_test.csv"

df1 = pd.read_csv(data_path)

def remove_stopwords(text):
    if isinstance(text, str):
        tokens = nltk.word_tokenize(text)
        stopwords = nltk.corpus.stopwords.words("english")
        filtered = [token for token in tokens if token.lower() not in stopwords]
        return ' '.join(filtered)
start = time.time()
df1["OriginalTweet"] = df1["OriginalTweet"].apply(remove_stopwords)
print(df1["OriginalTweet"])
end = time.time()
print(end - start)

0       TRENDING : New Yorkers encounter empty superma...
1       could n't find hand sanitizer Fred Meyer , tur...
2               Find protect loved ones # coronavirus . ?
3       # Panic buying hits # NewYork City anxious sho...
4       # toiletpaper # dunnypaper # coronavirus # cor...
                              ...                        
3793    Meanwhile Supermarket Israel -- People dance s...
3794    panic buy lot non-perishable items ? ECHO need...
3795    Asst Prof Economics @ cconces @ NBCPhiladelphi...
3796    Gov need somethings instead biar je rakyat ass...
3797    @ ForestandPaper members committed safety empl...
Name: OriginalTweet, Length: 3798, dtype: object
2.0049993991851807


In [2]:
!nvidia-smi

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.
